# Food Scrap Drop-Off Locations in NYC

The file contains locations, hours, and other information about food scrap drop-off sites in New York City & I have taken the latest version from the website - https://dev.socrata.com/foundry/data.cityofnewyork.us/if26-z6xq

ChatGPT (version 3.5) is able to provide some of the information needed. I.e if you ask to get a list of free food services on the eastside it will provide a list in the following form:

```About The Data

Number of records - 591 & Number of columns 27

```

As a Custom chatbot I would expect my chatbot to answer questions & give me response based on the contextual information available in the file.

## Setup

In [51]:
import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "open_ai_key"

## Data Read

In [25]:
df=pd.read_csv("Food_Scrap_Drop-Off_Locations_in_NYC.csv")
print(df.shape)

(591, 27)


In [52]:
df.head(1)

,Borough,NTAName,SiteName,SiteAddr,Hosted_By,Open_Month,Website
0,Brooklyn,Bay Ridge,4th Avenue Presbyterian Church,"6753 4th Avenue, Brooklyn, NY 11220",4th Avenue Presbyterian Church,Year Round,NaN


## Identify relevant columns & keep them only in the Dataframe

In [27]:
df=df[['Borough', 'NTAName', 'SiteName', 'SiteAddr', 'Hosted_By', 'Open_Month',        'Website']]

In [53]:
def generate_embeddings(texts):
    response = openai.Embedding.create(
        input=texts,
        model="text-embedding-ada-002"  # Use the appropriate model for embeddings
    )
    print(response)
    return [item['embedding'] for item in response['data']]

context_texts = df.apply(lambda row: f"Borough: {row['Borough']}, NTA Name: {row['NTAName']}, Site Address: {row['SiteAddr']}, Hosted By: {row['Hosted_By']},Open Month :{row['Open_Month']}  Website: {row['Website']}", axis=1).tolist()

# Generate embeddings for your context data
#context_embeddings = generate_embeddings(context_texts)

In [30]:
import json
np.save('context_embeddings.npy', context_embeddings)
with open('context_texts.json', 'w') as f:
    json.dump(context_texts, f)

## Asking Questions without context


In [44]:
def test_api():
    try:
        response = openai.ChatCompletion.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Where should I do my food scarp for Manhattan?"}
            ]
        )
        
        print("Test API Response:", response['choices'][0]['message']['content'].strip())
    
    except Exception as e:
        print(f"An error occurred during the test: {e}")
MODEL_NAME='gpt-4-turbo'
# Run the test
test_api()


Test API Response: In Manhattan, you have several options for disposing of food scraps responsibly. Here are some choices:

1. **Curbside Composting (if available)**: New York City is expanding its curbside composting program, which offers a convenient way to compost food scraps. Check the NYC Department of Sanitation (DSNY) website to see if this service is available in your area.

2. **Food Scrap Drop-off Sites**: The NYC Compost Project maintains numerous food scrap drop-off sites throughout Manhattan. You can bring your food scraps to these locations during specified hours. Locations include farmers markets, community gardens, and other public spaces. A good resource for finding a drop-off site near you is the NYC Zero Waste website.

3. **Community Composting**: Some community gardens and other local organizations in Manhattan may accept food scraps for composting. This can be a great way to get involved with your community while recycling your kitchen waste.

4. **Compost Collect

## Asking Question with Context


In [49]:
from sklearn.metrics.pairwise import cosine_similarity

def get_most_similar_context(query, context_embeddings, context_texts):
    # Generate embedding for the query
    query_embedding = generate_embeddings([query])[0]
    
    # Calculate similarity scores
    similarities = cosine_similarity([query_embedding], context_embeddings)
    
    # Find the most similar context
    most_similar_idx = np.argmax(similarities)
    return context_texts[most_similar_idx]

def generate_response(question, context_embeddings, context_texts):
    try:
        # Find the most relevant context
        relevant_context = get_most_similar_context(question, context_embeddings, context_texts)
        
        
        # Format the prompt
        prompt = f"""
        Answer the question in case of questions related to Food scrap based on the context below, if it can't be answered using the context, say "I don't know".

        Context:

        {relevant_context}

        ---

        Question: {question}

        Food Scrap Details for Location <Borugh>
        - NTA Name: <Name>
        - Site Address: <Site addr>
        - Hosted By: <Hosted by >
        - Website: <Website>

        Answer:
        """
        
        # Call the OpenAI API to generate a response
        response = openai.ChatCompletion.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        
        
        # Check response structure
        if 'choices' not in response or len(response['choices']) == 0:
            raise ValueError("Invalid response format or empty choices")

        return response['choices'][0]['message']['content'].strip()
    
    except KeyError as e:
        print(f"KeyError: {e}")
        raise
    
    except ValueError as e:
        print(f"ValueError: {e}")
        raise
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise




In [50]:
generate_response("Where should I do my food scarp for Manhattan?", context_embeddings, context_texts)

{
  "object": "list",
  "data": [
    {
      "object": "embedding",
      "index": 0,
      "embedding": [
        -0.0012531776446849108,
        -0.007355722598731518,
        0.020472373813390732,
        0.0014244248159229755,
        -0.0026679423172026873,
        0.0012522995239123702,
        -0.01944664865732193,
        -0.005933054257184267,
        -0.018322564661502838,
        -0.031137118116021156,
        0.02144189551472664,
        -0.005659058690071106,
        -0.010348595678806305,
        0.018266359344124794,
        -0.014247761107981205,
        -0.019573107361793518,
        0.02016325108706951,
        0.007180084474384785,
        -0.015245385468006134,
        -0.02163861133158207,
        -0.024055391550064087,
        0.019376393407583237,
        0.020753394812345505,
        -0.019797924906015396,
        0.0031158195342868567,
        0.009076976217329502,
        0.018322564661502838,
        -0.0009958678856492043,
        -0.010292391292750835,
   

'Food Scrap Details for Location Manhattan\n- NTA Name: Midtown South-Flatiron-Union Square\n- Site Address: Madison Avenue & E 25 Street\n- Hosted By: GrowNYC\n- Website: grownyc.org/compost'

### As we can see the answers change as per the context given & its mportant to make your custom chatbot